In [ ]:
#Basic AstroCV example 3
#Detect galaxies in online images
#Optimal image ratio = 4:3 (ratio of traning images)
#Detects galaxies in online images, then crops them so they have a 4:3 ratio, and compares number of galaxies detected in the cropped zone


import sys     
import pyyolo
import numpy as np
from PIL import Image, ImageEnhance
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import time
import urllib
import os.path

In [ ]:
if sys.version_info[0] >= 3:
    from urllib.request import urlretrieve
else:
    # Not Python 3 - today, it is most likely to be Python 2
    # But note that this might need an update when Python 4
    # might be around one day
    from urllib import urlretrieve
    
## This code imports urlretrieve from the right library, since they're different for Python versions 2 and  3
    
##code from https://stackoverflow.com/questions/17960942/attributeerror-module-object-has-no-attribute-urlretrieve

In [ ]:
darknet_path = './data/darknet' #darknet path
datacfg = '../sdss.data' #relative to darknet path
cfgfile = '../sdss.cfg' #relative to darknet path
weightfile = '/mnt/data/astrocv/galaxy_sdss_hic.weights'
filename = 'writable/tempImageDownload.jpg' #relative to galaxy_detection path


##Three images with bad ratios

#url = "https://www.nasa.gov/sites/default/files/thumbnails/image/hubble_friday_03112016.jpg" #square image
#url = "http://wwwcdn.skyandtelescope.com/wp-content/uploads/a-galaxy-cluster-1.jpg" #tall image
url = "https://i.kinja-img.com/gawker-media/image/upload/s--RkydXzP0--/c_scale,fl_progressive,q_80,w_800/t4eipzajwxxfp3retytv.jpg" #wide image


if not os.path.isfile(filename):
    urlretrieve(url,filename)
thresh = 0.1  #detection threshold try 0.02 & 0.1
hier_thresh = 0.5

In [ ]:
##The following code crops the image to a good image ratio.
##It won't crop the image if the ratio is ~4:3

img = Image.open(filename)
width, height = img.size
ratio=width/height

cropped = False
if abs(ratio-(4/3))>0.05:
        cropped = True
        if ratio<=(4/3):
                fheight=int(width*(3/4))
                img2 = img.crop((0, 0, width, fheight))
        else:
                fwidth = int(height*4/3)
                img2 = img.crop((0, 0, fwidth, height))
         
        img2.save("writable/tempImageDownload2.jpg")
        filename2="writable/tempImageDownload2.jpg"
        
        

In [ ]:
t1=time.time()
pyyolo.init(darknet_path, datacfg, cfgfile, weightfile) #init and load network
print('Initialization time = %5.3f seconds'%(time.time()-t1))
t1=time.time()
outputs = pyyolo.test(filename, thresh, hier_thresh, 0) #load image and process
print('Image processing time = %5.3f seconds'%(time.time()-t1))

asd=0

for output in outputs:
        asd+=1
        print(output)
        
print("%d images detected"%asd)
pyyolo.cleanup()


In [ ]:
##Here we process the cropped image, if cropped

if cropped:
    t2=time.time()
    pyyolo.init(darknet_path, datacfg, cfgfile, weightfile) #init and load network
    print('Initialization time = %5.3f seconds'%(time.time()-t2))
    t2=time.time()
    outputs2 = pyyolo.test(filename2, thresh, hier_thresh, 0) #load image and process
    print('Image processing time = %5.3f seconds'%(time.time()-t2))

    asd=0

    for output in outputs2:
            asd+=1
            print(output)

    print("%d images detected"%asd)
    pyyolo.cleanup()

In [ ]:
#plot image and detections
#plot red line showing where the crop was made (always above or at the left side of the line)


img = Image.open(filename)
contrast = ImageEnhance.Contrast(img)
img2 = contrast.enhance(1)
fig,ax = plt.subplots(figsize=(15,12))
plt.axis('off')
plt.tight_layout(pad=0)
plt.imshow(img2)
ax.set_aspect('equal')

inliers = 0
for output in outputs:
        r=output['right']
        l=output['left']
        t=output['top']
        b=output['bottom']
        if cropped:
                if width/height<4/3:
                        if b<fheight:
                                inliers+=1
                else:
                        if r<fwidth:
                                inliers+=1

                                
                                
        rect = patches.Rectangle((l-4,t-3),r-l+8,b-t+4,linewidth=1,edgecolor='b',facecolor='none')      
        ax.add_patch(rect)
        ax.annotate(output['class'],(l-7,t-19),color='w',fontsize=16)
        


if cropped:
        if width/height<4/3:
                plt.axhline(fheight,color='red')
        else:
                plt.axvline(fwidth,color='red')

plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0) 
plt.show()


In [ ]:
#plot image and detections for cropped image
img = Image.open(filename2)
contrast = ImageEnhance.Contrast(img)
img2 = contrast.enhance(1)
fig,ax = plt.subplots(figsize=(15,12))
plt.axis('off')
plt.tight_layout(pad=0)
plt.imshow(img2)
ax.set_aspect('equal')


for output in outputs2:
        r=output['right']
        l=output['left']
        t=output['top']
        b=output['bottom']
           
        rect = patches.Rectangle((l-4,t-3),r-l+8,b-t+4,linewidth=1,edgecolor='b',facecolor='none')      
        ax.add_patch(rect)
        ax.annotate(output['class'],(l-7,t-19),color='w',fontsize=16)
        


plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0) 
plt.show()

In [ ]:
print("In the raw image, %d galaxies were found in the area to be cropped"%inliers)
print("In the cropped image, %d galaxies were found"%asd )

In [ ]:
os.remove("writable/tempImageDownload.jpg")
os.remove("writable/tempImageDownload2.jpg")
##deletes temp images after downloading

In [ ]:
%%javascript
IPython.notebook.kernel.restart();